## Morning practical 1 day 6

Last week you traded some of your precious little time on this planet to painstakingly implement gradient descent for linear regression , logistic regression and neural networks yourself. Or you attempted to. Turns out ~that's a sucker's game~ that's unnecessary, because extremely cool and knowledgeable people have already put together a whole library that implements all sorts of ML algorithms, which allows you to train classifiers with very little code. This library is [scikit-learn](https://scikit-learn.org/stable/index.html). Here you will learn how to work with it. Rather than re-inventing the wheel, I will assign you specific parts of an excellent online tutorial to follow, interspersed with one exercise here. If you cannot grasp parts of the online tutorial, please let me know. Note that, for the tutorial I link to, answers are always available on the next page, so you should look for them there rather than here if you are stuck. 

In [ ]:
#run this cell to set things up
import ipywidgets as widgets, numpy as np, pandas as pd
from numpy.random import default_rng
%matplotlib inline
import matplotlib.pyplot as plt
import math
import seaborn as sns
from IPython.display import display, Markdown
import sklearn
import itertools
from sklearn.datasets import make_blobs

In [ ]:
# Used to make the train and test set. None of your concern!

# from sklearn.model_selection import train_test_split
# censusData = pd.read_csv("adult-census-numeric.csv")
# X_train, X_test = train_test_split(censusData, test_size = 0.2, random_state = 42, stratify = censusData["class"])

# X_test.to_csv("adult-census-numeric-test.csv")
# X_train.to_csv("adult-census-numeric.csv")

## The first simple sklearn steps

Go [here]() and follow the steps. Note that, for us, the data is directly in the working directory (so use `pd.read_csv('adult-census-data.csv')`). Over to you:

In [ ]:
# your answer here

## Step 2

Just continue on with the second step of the tutorial [here](https://inria.github.io/scikit-learn-mooc/python_scripts/02_numerical_pipeline_ex_00.html) where you see how to instantiate classifiers differently.

In [ ]:
# your answer here

## Step 3

Now, you will use scaling to scale the numerical features, and also combine the training and scaling in a _pipeline_. This is very important: when scaling with a train and a test set, you want to apply the scaling only to the train set, and then use its mean and variance to transform the test set. Otherwise, you use some characteristics of the test set to train your classifier (embedded in the information in the mean and variance) and your generalisation error will not be as independent as you want it to be.

Go [here](https://inria.github.io/scikit-learn-mooc/python_scripts/02_numerical_pipeline_scaling.html)

In [ ]:
# your answer here

## Consolidating what you've learned so far

Let's now switch to a small local assignment. It is up to you to implement linear regression (with and without regularisation) on the California housing dataset, which you'll have to import from sklearn by searching for it in their documentation. **You will probably need to import some modules from sklearn**.

* Load the California housing dataset (look in [sklearn.datasets](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets)).
* Print exactly what the targets mean (what are the units of what you're predicting?), what the number of samples is, and use `DataFrame.describe()` to see the mean and distribution of the features (you'll need to convert to a DataFrame for that, using `pd.DataFrame()`).
* Split the normalised data into train and test sets ( 80%, 20%, look [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)).
* Normalise both features and the target to have zero mean and unit variance. Use [StandardScaler](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling). You've just learned about pipelines for correctly 'fitting' your scaling to the train set and then applying it with the 'learned' mean and variance on the test set. **Use this!**
* Train linear regression without regularisation, and test on the test set. Print the $R^2$ (use `.score`). Also print the learned parameters/weights (use `.coef_`).
* Train linear regression with L2 regularisation (penalising the sum of squares of the parameters). This is called Ridge regression. Set its $\lambda$ (which is confusingly called alpha) to 1.  Test on the test set and print the $R^2$ (use `.score`). Also print the learned parameters/weights (use `.coef_`).


Hints:
* It seems like a lot of steps, but it's all mostly one-liners in scikit-learn.
* Search the documentation for the right predictor or functionality. Look for Ridge regression and/or linear regression  on the scikit-learn page and you will find what you need. You'll have to learn to navigate it anyway!
* If you get some dimensionality problems, do exactly what the error message says and your problems will melt away.
* If you don't know what $R^2$ is, see [this](https://www.youtube.com/watch?v=2AQKmw14mHM).

In [ ]:
from sklearn import datasets
# your answer here







## Moving on to cross-validation

Now go to the part of the tutorial on doing cross-validation in scikit-learn [here](https://inria.github.io/scikit-learn-mooc/python_scripts/02_numerical_pipeline_cross_validation.html). This is where the power of the pipeline _really_ kicks in: to do a correct cross-validation, you have to normalise the training data on the k-1 folds, and then use the mean and variance of those folds to scale the k-th fold. A pipeline takes care of all this for you.

In [ ]:
# your answer here

## Optimising hyperparameters with GridSearch

First let's make sure that you understand how to use grid search, which is the brute-force way of optimising  hyperparameters by just training a classifier for every possible combination of them (from a set you give), and then selecting the combination that performs best. Follow [this part](https://inria.github.io/scikit-learn-mooc/python_scripts/parameter_tuning_grid_search.html) of the tutorial to use the grid search function and do a bit more sophisticated pre-processing.

In [ ]:
# your answer here

## Nested cross-validation

Above, you used the naive approach to optimising hyperparameters. The tutorial already warned that this is not completely correct. I already told you what is correct: the heading of this text block. Make sure you understand the explanation below, then move on to [this part](https://inria.github.io/scikit-learn-mooc/python_scripts/cross_validation_nested.html) of the tutorial to implement correct hyperparameter selection for a model. After this, you're done and know enough of the basics to get going.

**Hyperparameter optimisation and nested cross-validation** <br>
Let's say we want to pick our learning rate. The naive approach would be to try 5 learning rates (say 0.001, 0.01, 0.1, 1, 10). You could, in your cross-validation, train 5 classifiers on each fold, one with each learning rate, and look at their average performance on the test folds. Then you could pick the $\alpha$ that's best and be done with it. **That's wrong**. This is because you pick which learning rate is best based on how it performs on the test set: so you're choosing the hyperparameters that do best on your 'unseen' test set, and then reporting that average performance. You will hence be overconfident!

So what to do? Well: nested-cross validation. You make k-1 folds to train on. _Within_ those k-1 folds, you _again_ divide into folds, say l folds. You train 5 classifiers on each of the l-1 folds, one for each value of $\alpha$. Then you test on the l-th fold. The $\alpha$ that perform best on average is the one you pick from among these sub-folds. Then, you move one level up again, and train using that alpha on the whole _upper_ fold of size k-1, and test using it on the k-th fold. Here is an image to further clarify:
<img src="nested-crossval.png" alt="Drawing" style="width: 760px;"/>

In the end, the whole goal is simple: you want to pick your hyperparameters based on _one_ unseen part of the data, and then measure your generalisation performance on some _other_ unseen part of the data that the hyperparameters were not fitted to, so you don't get a positive bias of your generalisation potential. 

**Picking final hyperparameters for your full model** <br>
Now there's one little detail: this means you end up with k folds that were each, potentially, trained with different hyperparameters. Perhaps on fold 3, an $\alpha$ of 0.01 was best, but on fold 5 one of 0.001 was best. For your _final model_ you could then pick either the hyperparameter that is picked in most of you k folds, or you could take an average.

Remember: you're supposed to go [here](https://inria.github.io/scikit-learn-mooc/python_scripts/cross_validation_nested.html).

In [ ]:
# your answer here

## Note on dimensionality reduction and pipelines

As I showed in the lectures, you can think of dimensionality reduction as just another part of preprocessing of your data, which should _also_ be performed in a pipeline: You dimension-reduce the training samples, transform the test samples to be in that feature space, and then predict. You could even try different scaling algorithms, and see _which_ scaling algorithm to use as a hyperparameter, just as _which model_ you use (say logistic regression or a tree-based model) is, in fact, a hyperparameter. See one of my last slides for an example of implementing nested hyperparameter optimalisation using different dimensionality reduction tools, (and also trying different parameters _in_ these different dimensionality reduction tools). **This would be a _huge_ headache if you had to do it yourself, but pipelines take care of it for you!**

## What I want you to remember here:
* Some practical skills: how sklearn models work (`.fit()`, `.predict`, etc.); how to make sure scaling is combined correctly with cross-validation or even nested-cross validation using a pipeline; how to perform nested cross-validation.
* That [this tutorial](https://inria.github.io/scikit-learn-mooc/index.html) exists, so that you can use it to look up things you want more information on during the project.
* Use those pipelines!

## Extra resource
[Here's a guy who goes through all the stuff you've done here in a youtube video](https://www.youtube.com/watch?v=XvnkUg1yVmk) where you can see what he's doing and hear why.

## Survey
Doubting whether or not to fill out the survey? I'll let my friends Shia LaBeouf and Schmoyoho [do the talking for me](https://youtu.be/UhRXn2NRiWI?t=45).

[Much survey, so wow](https://docs.google.com/forms/d/e/1FAIpQLSdmRCFUfpeYNch2V4sQInGoM2liJ15XGYxNmWoBpc5QX3kIqw/viewform?usp=sf_link)